In [1]:
from utils.general import get_logger
from utils.test_env import EnvTest
from q1_schedule import LinearExploration, LinearSchedule
from dqn import DQN
from configs.q3_nature import config

In [2]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [3]:
env = EnvTest((80, 80, 1))
model = DQN(env, config)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


From /usr/local/lib/python2.7/site-packages/tensorflow/python/ops/clip_ops.py:113: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


## q nets and update test

In [4]:
s = tf.ones([1, 80, 80, 4], dtype=tf.float32)
sp = tf.ones([1, 80, 80, 4], dtype=tf.float32)

In [5]:
def is_equal(x, y):
    is_equal = tf.equal(x, y)
    is_equal = tf.logical_not(is_equal)
    is_equal = tf.cast(is_equal, tf.int32)
    is_equal = tf.reduce_sum(is_equal)
    is_equal = tf.equal(is_equal, 0)
    return is_equal

In [7]:
q_test = model.get_q_values_op(s, scope="q_test", reuse=False)
target_q_test = model.get_q_values_op(sp, scope="target_q_test", reuse=False)
update_target_op = model.add_update_target_op("q_test", "target_q_test")

In [8]:
q_test_var_lst = tf.get_collection(
    tf.GraphKeys.TRAINABLE_VARIABLES,
    "q_test")
target_q_test_var_lst = tf.get_collection(
    tf.GraphKeys.TRAINABLE_VARIABLES,
    "target_q_test")

In [9]:
assert len(q_test_var_lst) == len(target_q_test_var_lst), \
  "number of variables in q and target_q differ"

In [15]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
distance_before_lst = []

# check difference before the update
for idx in range(len(target_q_test_var_lst)):
    # skip bias, since they are intialized with 0's
    if 'bias' in q_test_var_lst[idx].name:
        continue
    distance_np = (sess.run(tf.norm(
        q_test_var_lst[idx] -
        target_q_test_var_lst[idx]
    )))
    distance_before_lst.append(distance_np)
    
assert np.mean(distance_before_lst) != 0., \
  'q and taget_q initialized with the same weights'

# perform update
sess.run(update_target_op)

# check difference after the update
distance_after_lst = []
for idx in range(len(target_q_test_var_lst)):
    # skip bias, since they are intialized with 0's
    if 'bias' in q_test_var_lst[idx].name:
        continue
    distance_np = (sess.run(tf.norm(
        q_test_var_lst[idx] -
        target_q_test_var_lst[idx]
    )))
    distance_after_lst.append(distance_np)
assert np.mean(distance_after_lst) == 0., \
  'q and taget_q weights are different after the update'

## loss test

In [36]:
model.env.action_space.n = 3

In [50]:
a_np = np.array([0, 1], dtype=np.int64)
q_test_np = np.array([
    [0.1, 0.2, 0.3],
    [0.6, 0.5, 0.4]   
], dtype=np.float32)
target_q_test_np = np.array([
    [1.1, 1.2, 1.3],
    [1.6, 1.5, 1.4]   
], dtype=np.float32)
r_np = np.array([1., 10.], dtype=np.float32)
done_mask_np = np.array([0., 1.], dtype=np.float32)
num_actions = 3


In [51]:
q_test_np.dtype

dtype('float32')

In [52]:
q_test_placeholder = tf.placeholder(
    tf.float32,
    shape=[2, 3])
q_target_test_placeholder = tf.placeholder(
    tf.float32,
    shape=[2, 3])

loss_tf = model.add_loss_op(
    q_test_placeholder,
    q_target_test_placeholder)

loss_np = sess.run(
    loss_tf,
    feed_dict={
        q_test_placeholder: q_test_np,
        q_target_test_placeholder: target_q_test_np,
        model.a: a_np,
        model.r: r_np ,
        model.done_mask: done_mask_np
    }
)

In [59]:
loss_np

47.516483

In [60]:
loss_agg = r_np + (1. - done_mask_np) * \
    model.config.gamma * target_q_test_np.max(axis=1) - q_test_np[[0, 1], a_np]

In [61]:
np.mean(loss_agg ** 2)

47.516483